# Разработка и поддержка библиотеки ZenitAI

## Общие положения

1. ZenitAI - это Python-библиотека
2. Версионируется с помомощью git и setuptools
3. Проект использует автоматические форматтеры кода и линтеры

### Расположение репозитория

Основной репозиторий с проектом хранится в общей папке:  
    
    K:\ДМиРТ\Управление моделирования\#Zenit.ai\git_repos\zenitai-lib.git


### Требуемые пакеты Python

- python
- numpy
- pandas
- matplotlib
- scikit-learn
- openpyxl
- ipython
- tqdm
- scipy

### Ссылки на источники

- Настройка git на сервере: 
    [ссылка](https://git-scm.com/book/ru/v2/Git-%D0%BD%D0%B0-%D1%81%D0%B5%D1%80%D0%B2%D0%B5%D1%80%D0%B5-%D0%9F%D1%80%D0%BE%D1%82%D0%BE%D0%BA%D0%BE%D0%BB%D1%8B)
- Блог-пост про git-hooks и автоформаттеры:
    [ссылка](https://ternaus.blog/tutorial/2020/08/28/Trained-model-what-is-next.html)


## Структура решения


Папка с проектом на данный момент выглядит так
![tree](img/tree.PNG)sadsad

Краткое описание папок:  
- `build` - содержит файлы, необходимые для сборки пакета
- `dist` - содержит архивы со всеми собранными версиями библиотеки - из этих архивов можно устанавливать через `pip install`
- `docs` - содержит файлы, необходимые для автоматического создания документации. Документация генерируется на основе docstrings функциях/классах и этих файлах с помощью библиотеки _Sphinx_
    - `tutorial` - здесь лежит этот гайд
- `zenitai` - папка с исходным кодом проекта  
   - `experiment` - класс Experiment для контроля экспериментов и запусков моделей
   - `transform` - инструменты для преобразования данных (в частности - WoE-трансформер)
   - `utils` - функции, которым пока не нашлось отдельного раздела: функции-хелперы, функции для вычисления метрик (вроде расчета Gini на основе ROC AUC и отрисовки самих ROC-кривых)
- `ZenitAI.egg-info` - служебные данные, которые создаются при установке пакета

Краткое описание файлов:
- `.gitignore` - список файлов, папок и шаблонов, которые будут игнорироваться гитом при версионировании
- `pyproject.toml` - содержит конфиг для автоматического форматтера кода `black`
- `setup.cfg` - конфиг для модуля `setuptools` и для линтера `flake8`
- `setup.py` - тоже конфиг для модуля `setuptools`

Кроме того, есть еще файлы с конфигами:
- `.pre-commit-config.yaml` - конфиг для git-хуков (git-hooks). Нужен для автоматического форматирования, линтинга (проверки на соответствие стиля кода) и проверки кода при коммитах в git. При коммите будет предпринята попытка отформатировать код, если она будет неудачной, коммит сделать не получится до устранения замечаний. Это нужно, чтобы поддерживать код в читабельном состоянии и для соблюдения некоторых правил оформления в случае совместной работы 

## Начало работы

Предлагаю организовать работу над дэшбордами в таком формате:
- Разработка и отладка функционала библиотеки: локально на рабочем ПК
- Сборка библиотеки под новой версией 
- Сохранение результатов в git-репозитории в общей папке
- Подтягивание обновленного статуса проекта в личную папку пользователя `dashboards_dev`


**Более подробно:**  

0. Установить git на рабочий ПК: 
    [ссылка для скачивания](https://git-scm.com/download/win)
1. Склонировать репозиторий на рабочий ПК: в терминале (в командной строке, или в git bash, или в терминале IDE) набрать  
        git clone "K:\ДМиРТ\Управление моделирования\#Zenit.ai\zenitai" <путь к папке, где будет лежать проект>
2. Создать и активировать виртуальное окружение для проекта:
        conda create -n <environment_name> 
        conda activate dashboards
        conda install pandas numpy <...>
        cd <полный путь к папке>     #перемещение консоли в папку проекта
4. Установить пакет в режиме редактирования:
        pip install -e .
   Это позволит импортировать подмодули проекта в самих файлах, а также установит в окружение (почти) ве нужные библиотеки
5. Внести нужные изменения  
...
6. Проверить, что локально функционал работает так, как задумано
7. Отправить изменения в центральный репозиторий
        git add .
        git commit -m '<текст сообщения>'
        git push origin master
8. Скопировать изменения в папку на общем сервере
    - Открыть в проводнике папку _K:\ДМиРТ\Управление моделирования\#Zenit.ai\zenitai_
    - Кликнуть правой кнопкой по пустому месту в папке, выбрать пункт _"git bash here"_
        ![git_bash](img/git_bash.PNG)
    - Выполнить команду для получения изменений  
            git pull origin master
9. Приложение должно обновиться само через 10-20 секунд

## Общая логика работы приложения

0. В центре большинства операций находится файл `models_and_features.xlsx` - реестр моделей и фичей
    - В файле хранится информация о моделях, сроках их эксплуатации, системах и фичах
    
1. Cначала подготавливаются данные для визуализации и складываются в csv-файлы (скрипт `src/data/prepare_data.py`)
    - Для целей разработки используются предварительно выгруженные данные из витрин DM_RISK  
    - Для целей нормальной эксплуатации предлагается использовать витрины напрямую и подготавливать данные раз в сутки (например)
        - Для этого нужен доступ с сервера к БД
    - В скрипте prepare_data.py предусмотрен глобальная переменная `MODE`, которая управляет способом загрузки данных - из csv или из витрин
    - Подключаться дэшбордом напрямую к витринам - не самая лучшая идея, потому что в таком случае скорость и стабильность обновления данных на дэшборде непредсказуема из-за рандомных обновлений витрин в DM_RISK
2. Подготовленные данные подхватываются скриптом для самого дэшборда (скрипт `src/app/app.py`)
    - в скрипте происходит загрузка предподготовленных данных, определение функций с колл-бэками dash и запуск самого сервера дэшбордов
3. Внешний вид определяется отдельным скриптом (`src/app/layout.py`)


## Запуск приложения

1. Убедиться, что окружение `dashboards` активировано
2. Запустить скрипт проложения через командную строку
        python <путь к файлу app.py>
   Например, если консоль открыта в папке проекта, то строка-приглашение в терминале должна выглядеть так:  
        (dashboards) C:\Users\s.kadochnikov\Desktop\Проекты\Отчетность>
   После ввода команды она должна выглядеть так:  
        (dashboards) C:\Users\s.kadochnikov\Desktop\Проекты\Отчетность>python src/app/app.py
3. если все прошло успешно, то после загрузки данных и их обработки дэшборд бдет доступен по адресу и порту, которые были указаны в файле `.env`. Если там они не указаны, то по умолчанию дэшборд развернется на адресе `127.0.0.1:8050`